In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("Final") \
    .getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/04 08:52:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [5]:
df = spark.read.parquet('./cleaned_dataset')

In [6]:
df.printSchema()

root
 |-- lei: string (nullable = true)
 |-- loan_type: integer (nullable = true)
 |-- loan_purpose: integer (nullable = true)
 |-- loan_amount: float (nullable = true)
 |-- interest_rate: float (nullable = true)
 |-- loan_term: integer (nullable = true)
 |-- action_taken: integer (nullable = true)
 |-- income: float (nullable = true)
 |-- applicant_age: string (nullable = true)
 |-- applicant_sex: integer (nullable = true)
 |-- applicant_credit_score_type: integer (nullable = true)
 |-- co_applicant_age: string (nullable = true)
 |-- co_applicant_credit_score_type: integer (nullable = true)
 |-- derived_msa_md: integer (nullable = true)
 |-- state_code: string (nullable = true)
 |-- county_code: string (nullable = true)
 |-- property_value: float (nullable = true)
 |-- total_units: integer (nullable = true)
 |-- occupancy_type: integer (nullable = true)



In [7]:
len(df.columns)

19

In [8]:
df.count()

7427

### 4. Train Test split

In [9]:
train_df, test_df = df.randomSplit(weights=[0.8, 0.2], seed=100)

In [10]:
train_df.count()

5915

In [11]:
test_df.count()

1512

### 5. Feature transformation

In [12]:
from pyspark.sql.types import StringType

In [13]:
from pyspark.ml.feature import StringIndexer, StandardScaler, VectorAssembler, FeatureHasher
from pyspark.ml import Pipeline

In [14]:
# Define categorical and numerical features
categorical_features = [field.name for field in train_df.schema.fields if field.dataType is StringType()]
numerical_features = [field for field in train_df.columns if field not in categorical_features]

continuous_numerical_features = ["loan_amount", "interest_rate", "loan_term", "income", "property_value"]
discrete_numerical_features = [item for item in numerical_features if item not in continuous_numerical_features]

In [15]:
categorical_features.remove('county_code')

In [16]:
categorical_features

['lei', 'applicant_age', 'co_applicant_age', 'state_code']

In [17]:
continuous_numerical_features

['loan_amount', 'interest_rate', 'loan_term', 'income', 'property_value']

In [18]:
discrete_numerical_features

['loan_type',
 'loan_purpose',
 'action_taken',
 'applicant_sex',
 'applicant_credit_score_type',
 'co_applicant_credit_score_type',
 'derived_msa_md',
 'total_units',
 'occupancy_type']

In [19]:
cat_indexed_features = [f"{cat}_indexed" for cat in categorical_features]

In [20]:
# perform Label encoding on categorical features
labelEncoder = StringIndexer(inputCols=categorical_features, 
                           outputCols=cat_indexed_features, handleInvalid="skip")

In [21]:
# perform feature hashing on 'county_code' as a lot of distinct values
hasher = FeatureHasher(inputCols=["county_code"], outputCol="county_code_hashed", numFeatures=1000)

In [22]:
# perform Standard scaling on continuous numerical features
numAssembler = VectorAssembler(inputCols=continuous_numerical_features, 
                            outputCol="con_num_features")

numScaler = StandardScaler(inputCol="con_num_features", outputCol="con_num_features_scaled")

In [23]:
# assemble all the features together
featureAssembler = VectorAssembler(inputCols=["con_num_features_scaled", "county_code_hashed"]+discrete_numerical_features+cat_indexed_features, outputCol='features')

In [24]:
# make the Pipeline
transformPipeline = Pipeline(stages = [labelEncoder, hasher, numAssembler, numScaler, featureAssembler])

In [25]:
# train it
transformPipeModel = transformPipeline.fit(train_df)

In [26]:
train_df = transformPipeModel.transform(train_df)

In [27]:
train_df

24/12/04 08:52:30 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


lei,loan_type,loan_purpose,loan_amount,interest_rate,loan_term,action_taken,income,applicant_age,applicant_sex,applicant_credit_score_type,co_applicant_age,co_applicant_credit_score_type,derived_msa_md,state_code,county_code,property_value,total_units,occupancy_type,lei_indexed,applicant_age_indexed,co_applicant_age_indexed,state_code_indexed,county_code_hashed,con_num_features,con_num_features_scaled,features
01J4SO3XTWZF4PP38209,1,1,465000.0,3.5,360,1,188.0,35-44,2,3,35-44,3,13820,AL,01117,495000.0,1,1,93.0,0.0,1.0,26.0,"(1000,[710],[1.0])","[465000.0,3.5,360...",[2.04958786384093...,"(1018,[0,1,2,3,4,..."
01J4SO3XTWZF4PP38209,1,2,5000.0,3.2904205,60,0,27.0,65-74,1,1,9999,10,32820,TN,47157,75000.0,1,1,93.0,4.0,0.0,20.0,"(1000,[611],[1.0])","[5000.0,3.2904205...",[0.02203857918108...,"(1018,[0,1,2,3,4,..."
01J4SO3XTWZF4PP38209,1,4,15000.0,0.98,120,1,58.0,55-64,2,7,65-74,9,25060,MS,28047,125000.0,2,1,93.0,3.0,6.0,36.0,"(1000,[149],[1.0])","[15000.0,0.980000...",[0.06611573754325...,"(1018,[0,1,2,3,4,..."
01J4SO3XTWZF4PP38209,1,31,25000.0,5.5,120,1,187.0,35-44,2,7,35-44,9,25620,MS,28035,405000.0,1,1,93.0,0.0,1.0,36.0,"(1000,[627],[1.0])","[25000.0,5.5,120....",[0.11019289590542...,"(1018,[0,1,2,3,4,..."
01J4SO3XTWZF4PP38209,1,31,45000.0,4.25,59,1,94.0,8888,4,9,9999,10,27140,MS,28049,75000.0,1,3,93.0,5.0,0.0,36.0,"(1000,[766],[1.0])","[45000.0,4.25,59....",[0.19834721262976...,"(1018,[0,1,2,3,4,..."
01J4SO3XTWZF4PP38209,1,31,205000.0,3.0,360,1,97.0,65-74,1,1,65-74,1,19300,AL,01003,375000.0,1,1,93.0,4.0,6.0,26.0,"(1000,[581],[1.0])","[205000.0,3.0,360...",[0.90358174642449...,"(1018,[0,1,2,3,4,..."
01J4SO3XTWZF4PP38209,1,32,105000.0,2.625,180,1,65.0,45-54,2,2,9999,10,25060,MS,28047,175000.0,1,1,93.0,1.0,0.0,36.0,"(1000,[149],[1.0])","[105000.0,2.625,1...",[0.46281016280279...,"(1018,[0,1,2,3,4,..."
01J4SO3XTWZF4PP38209,2,1,115000.0,3.99,360,0,94.0,8888,4,9,8888,9,99999,MS,28083,125000.0,1,1,93.0,5.0,4.0,36.0,"(1000,[791],[1.0])","[115000.0,3.99000...",[0.50688732116496...,"(1018,[0,1,2,3,4,..."
01J4SO3XTWZF4PP38209,2,1,315000.0,2.875,360,1,87.0,25-34,1,1,9999,10,32820,MS,28137,325000.0,1,1,93.0,2.0,0.0,36.0,"(1000,[744],[1.0])","[315000.0,2.875,3...",[1.38843048840837...,"(1018,[0,1,2,3,4,..."
0S8H5NJFLHEVJXVTQ413,1,32,285000.0,3.5,360,1,64.0,25-34,3,1,25-34,1,13380,WA,53073,355000.0,1,1,577.0,2.0,3.0,10.0,"(1000,[634],[1.0])","[285000.0,3.5,360...",[1.25619901332186...,"(1018,[0,1,2,3,4,..."


### 6. Models Prediction

In [28]:
from pyspark.ml.classification import LogisticRegression, LinearSVC, NaiveBayes, DecisionTreeClassifier, RandomForestClassifier, GBTClassifier, FMClassifier

In [29]:
# evaluate the model from 'y_true' and 'y_pred' 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [30]:
def evaluate_model(predictions, label_col='action_taken', prediction_col='prediction', raw_prediction_col='rawPrediction'):
    '''It returns classification evaluation metrics like accuracy, precision, f1, recall and roc'''
    
    # Initialize evaluators
    evaluator_accuracy = MulticlassClassificationEvaluator(labelCol=label_col, predictionCol=prediction_col, metricName='accuracy')
    evaluator_precision = MulticlassClassificationEvaluator(labelCol=label_col, predictionCol=prediction_col, metricName='weightedPrecision')
    evaluator_recall = MulticlassClassificationEvaluator(labelCol=label_col, predictionCol=prediction_col, metricName='weightedRecall')
    evaluator_f1 = MulticlassClassificationEvaluator(labelCol=label_col, predictionCol=prediction_col, metricName='f1')
    evaluator_roc = BinaryClassificationEvaluator(labelCol=label_col, rawPredictionCol=raw_prediction_col, metricName='areaUnderROC')

    # Calculate metrics
    accuracy = evaluator_accuracy.evaluate(predictions)
    precision = evaluator_precision.evaluate(predictions)
    recall = evaluator_recall.evaluate(predictions)
    f1_score = evaluator_f1.evaluate(predictions)
    roc_auc = evaluator_roc.evaluate(predictions)

    # Return all metrics as a dictionary
    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'roc_auc': roc_auc
    }

    return metrics

In [31]:
models = {
    'Logistic Regression': LogisticRegression(featuresCol='features', labelCol='action_taken'),
    'Support Vector Machine': LinearSVC(featuresCol='features', labelCol='action_taken'),
    # 'Naive Bayes': NaiveBayes(featuresCol='features', labelCol='action_taken'),
    'Factorization Machine': FMClassifier(featuresCol='features', labelCol='action_taken'),
    'Decision Tree': DecisionTreeClassifier(featuresCol='features', labelCol='action_taken', maxBins=2000),
    'Random Forest': RandomForestClassifier(featuresCol='features', labelCol='action_taken', maxBins=2000),
    'Gradient Boosting Trees': GBTClassifier(featuresCol='features', labelCol='action_taken', maxBins=2000),
}

In [32]:
for algo in models:
    print(f"========== {algo} ============")

    # Train the model
    model = models[algo]
    trained_model = model.fit(train_df)

    # Evaluate on Test data
    test_df_transformed = transformPipeModel.transform(test_df)
    test_predictions = trained_model.transform(test_df_transformed)

    results = evaluate_model(test_predictions)
    print("accuracy: {:.4f}".format(results['accuracy']))
    print("precison: {:.4f}".format(results['precision']))
    print("recall: {:.4f}".format(results['recall']))
    print("f1-score: {:.4f}".format(results['f1_score']))
    print("ROC: {:.4f}".format(results['roc_auc']))

    print('\n')

========== Logistic Regression ============


24/12/04 08:52:31 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/12/04 08:52:31 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


accuracy: 1.0000
precison: 1.0000
recall: 1.0000
f1-score: 1.0000
ROC: 1.0000


========== Support Vector Machine ============
accuracy: 0.9993
precison: 0.9993
recall: 0.9993
f1-score: 0.9993
ROC: 0.9998


========== Factorization Machine ============
accuracy: 0.9364
precison: 0.9361
recall: 0.9364
f1-score: 0.9358
ROC: 0.9572


========== Decision Tree ============
accuracy: 1.0000
precison: 1.0000
recall: 1.0000
f1-score: 1.0000
ROC: 1.0000


========== Random Forest ============
accuracy: 0.8208
precison: 0.8581
recall: 0.8208
f1-score: 0.7963
ROC: 0.9710


========== Gradient Boosting Trees ============
accuracy: 1.0000
precison: 1.0000
recall: 1.0000
f1-score: 1.0000
ROC: 1.0000


